In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from time import sleep
from selenium.webdriver.common.by import By
import pandas as pd
import pickle
import psutil
import numpy as np

from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton

In [2]:
import dotenv
import os

dotenv.load_dotenv()

user = os.getenv("BSKY_USER")
password = os.getenv("BSKY_PASS")

https://bsky.app/profile/did:plc:kkf4naxqmweop7dv4l2iqqf5/feed/verified-news  
https://bsky.app/search?q=s%26p+500  
https://deck.blue/  

In [3]:
opts = FirefoxOptions()
opts.add_argument("--width=400")
opts.add_argument("--height=800")
service = Service('/snap/bin/firefox.geckodriver')
#opts.add_argument('--headless')

In [4]:
try:
  nv.quit()
  del nv
except:
    pass
nv = webdriver.Firefox(options = opts, service=service)

nv.get('https://deck.blue/')
sleep(10)

In [5]:
script = """
var glassPane = document.querySelector('flt-glass-pane');
if (glassPane) {
    var shadowRoot = glassPane.shadowRoot;
    if (shadowRoot) {
        var semanticsPlaceholder = shadowRoot.querySelector('flt-semantics-placeholder');
        if (semanticsPlaceholder) {
            semanticsPlaceholder.click({force: true});
        } else {
            console.error('Elemento flt-semantics-placeholder não encontrado.');
        }
    } else {
        console.error('shadowRoot de flt-glass-pane não encontrado.');
    }
} else {
    console.error('Elemento flt-glass-pane não encontrado.');
}
"""

In [6]:
nv.execute_script(script)

In [7]:
inputs = nv.find_elements(By.TAG_NAME, 'input')

username_ipt = inputs[0]
password_ipt = inputs[1]

username_ipt.click()
username_ipt.send_keys(user)
sleep(3)

In [8]:
for _ in range(20):
    wait = WebDriverWait(nv, 10)
    wait.until(EC.element_to_be_clickable((By.TAG_NAME, 'input')))

    try:
        # Tentar clicar no elemento
        password_ipt.click()
        sleep(1)
    except ElementClickInterceptedException:
        # Usar JavaScript para clicar no elemento se o clique normal falhar
        nv.execute_script("arguments[0].click();", password_ipt)
# Enviar a senha para o campo de entrada
password_ipt.send_keys(password)
sleep(1)
password_ipt.send_keys(password)

In [9]:
# click on login
login_btn = nv.find_element(By.XPATH, '//flt-semantics[@aria-label="Login"]')
login_btn.click()

sleep(10)

In [31]:
nv.execute_script(script)

In [11]:
feed = nv.find_element(By.XPATH, '//flt-semantics[@id="flt-semantic-node-49"]')
# find child element flt-semantics-container
#container1 = feed.find_element(By.XPATH, '//flt-semantics-container')

# get all flt-semantics-container child elements
# and select last one
elementos = nv.find_elements(By.XPATH, '//flt-semantics-container')
elementos_filtrados = []
for elemento in elementos:
    style = elemento.get_attribute('style')
    if ('position: absolute;' in style and
        'pointer-events: none;' in style and
        'top: 10px;' in style and
        'left: 0px;' in style):
        elementos_filtrados.append(elemento)

_feed = elementos_filtrados[0]
soup = BeautifulSoup(_feed.get_attribute('innerHTML'), 'html.parser')
# get all aria-label values
labels = [x.attrs['aria-label'] for x in soup.find_all('flt-semantics') if 'aria-label' in x.attrs]

parent_labels = []
child_labels = []
for label in labels[1:]:
    if 'Reply' in label:
        #print(child_labels)
        parent_labels.append(child_labels)
        child_labels = []
    else:
        child_labels.append(label)    

print(parent_labels)


#_post = _feed.find_element(By.XPATH, f'{_feed.get_attribute("xpath")}//flt-semantics-container')
#_post_card = _post.find_element(By.XPATH, '//flt-semantics-container//flt-semantics//flt-semantics-container')
#_soup = BeautifulSoup(_post.get_attribute('innerHTML'), 'html.parser')

#print(_soup.prettify())
#elhtml = elementos_filtrados[0].find_elements(By.TAG_NAME, 'flt-semantics')[0].get_attribute('outerHTML')

#soup = BeautifulSoup(elhtml, 'html.parser')
# get all aria-label values
#labels = [x.attrs['aria-label'] for x in soup.find_all('flt-semantics') if 'aria-label' in x.attrs]
#labels

[['🦋Jenny Q Ta', '@jennyqta.bsky.social', '2023/06/09 13:26:10\n1y', 'Interesting 🤨🧐🤔\n\nDow, Nasdaq and S&P 500 are rising but most stocks are falling\n\n', 'stocks.apple.com\nDow, Nasdaq and S&P 500 are rising but most stocks are falling', 'Show menu'], ['BizToc', '@biztoc.com', '2023/06/09 04:43:34\n1y', "Asia markets rise as S&P 500 hits new 2023 high, China's producer price index falls\n#asiapacificregion #metalgear #silenthill #fastretailing #mitsui #daikinindustries\n\n➡️ Now on ", ' — ', 'Show menu'], ['Sketchy News', '@sketchynews.net', '2023/06/09 00:30:14\n1y', " (finance.yahoo.com) Fundstrat head of research Tom Lee believes investors aren't bullish enough on a rally in the S&P 500 to continue throughout 2023.", 'Show menu'], ['Blurry TV Headlines', '@blurrytvheadlines.numbergoup.com', '2023/06/08 17:00:58\n1y', 'S&P 500 RISES 20% FROM OCTOBER LOW TO ENTER BULL MARKET', 'Show menu'], ['BizToc', '@biztoc.com', '2023/06/08 04:46:12\n1y', 'S&P 500’s Climb Toward Bull Market Th

In [28]:
unique_posts = []
#container_children = container.find_elements(By.XPATH, '//flt-semantics')
feed = None

try:
    elementos = nv.find_elements(By.XPATH, '//flt-semantics-container')
    for elemento in elementos:
        style = elemento.get_attribute('style')
        if ('position: absolute;' in style and
            'pointer-events: none;' in style and
            'top: 10px;' in style and
            'left: 0px;' in style):
            feed = elemento
            break
except Exception as e:
    print(f'Error in feed choice: {e}')


feed_scroll = None
try:
    elementos_ = nv.find_elements(By.XPATH, '//flt-semantics')
    for elemento in elementos_:
        style = elemento.get_attribute('style')
        if ('position: absolute;' in style and
            'pointer-events: none;' in style and
            'width: 350px;' in style and
            'height: 659px;' in style):
            feed_scroll = elemento
            break
except Exception as e:
    print(f'Error in feed scroll choice: {e}')

In [29]:
feed_scroll_html = feed_scroll.get_attribute('outerHTML')
feed_scroll_soup = BeautifulSoup(feed_scroll_html, 'html.parser')

# find feed_scroll id
feed_scroll_id = feed_scroll_soup.find('flt-semantics').attrs['id']
feed_scroll_id

'flt-semantic-node-49'

In [34]:
scroll_count = 100000
for _ in range(scroll_count):
    if psutil.virtual_memory().percent >= 85:
        print('Memory usage is too high. Exiting...')
        print(f'last unique post: {unique_posts[-1]}')
        random_nbr = str(np.random.randint(0, 100000))
        with open(f'sp500_bskyhtml-exit_{random_nbr}.pkl', 'wb') as f:
            pickle.dump(unique_posts, f)
        break
    try:
        feed_scroll = nv.find_element(By.XPATH, '//flt-semantics[@id="flt-semantic-node-125"]')
        #containers = feed_scroll.find_elements(By.XPATH, '//flt-semantics-container')

        soup = BeautifulSoup(feed.get_attribute('innerHTML'), 'html.parser')
        labels = [x.attrs['aria-label'] for x in soup.find_all('flt-semantics') if 'aria-label' in x.attrs]
        
        parent_labels = []
        child_labels = []
        for label in labels[1:]:
            if 'Reply' in label:
                #print(child_labels)
                parent_labels.append(child_labels)
                child_labels = []
            else:
                child_labels.append(label) 

        for post in parent_labels:
            if post not in unique_posts:
                unique_posts.append(post)

        print(f'Iteração: {_}')
        print(f'Post count: {len(unique_posts)}')

        feed_scroll.send_keys(Keys.PAGE_DOWN)
        sleep(1)
    except Exception as e:
        print(f'Error in scrolling: {e}')
        break
with open('sp500_bskyhtml.pkl', 'wb') as f:
    pickle.dump(unique_posts, f)

Iteração: 0
Post count: 5
Iteração: 1
Post count: 8
Iteração: 2
Post count: 12
Iteração: 3
Post count: 15
Iteração: 4
Post count: 17
Iteração: 5
Post count: 20
Iteração: 6
Post count: 24
Iteração: 7
Post count: 26
Iteração: 8
Post count: 28
Iteração: 9
Post count: 31
Iteração: 10
Post count: 35
Iteração: 11
Post count: 38
Iteração: 12
Post count: 39
Iteração: 13
Post count: 39
Iteração: 14
Post count: 39
Iteração: 15
Post count: 39
Iteração: 16
Post count: 39
Iteração: 17
Post count: 39
Iteração: 18
Post count: 39
Iteração: 19
Post count: 39
Iteração: 20
Post count: 39
Iteração: 21
Post count: 39
Iteração: 22
Post count: 39
Iteração: 23
Post count: 39


KeyboardInterrupt: 

In [35]:
len(unique_posts)

39

In [38]:
unique_posts[-1]

['*companies that make up 20% of S&P 500* \n\nwow look how stupid these companies are they can’t do anything right',
 '2023/05/09 12:00:53\n@metallurgist.bsky.social\n1y\ncontent like this from their editor confirms to me that the verge has no meaningful commentary to be made about tech',
 'Show menu']

In [39]:
with open('sp500_bskyhtml090523.pkl', 'wb') as f:
    pickle.dump(unique_posts, f)